In [1]:
from PyPDF2 import PdfFileReader
import pandas as pd
import numpy as np
import camelot
import tabula

In [13]:
file_path = "ABB-BRSR Report.pdf"
pdf_file = open(file_path,"rb")

In [14]:
reader = PdfFileReader(pdf_file, strict=False)

In [16]:
num_pages = reader.numPages
print(num_pages)

42


In [22]:
## list of Sections to get search pattern and extract tables section wise
#hdfc sections
#sections = ['SECTION A: GENERAL DISCLOSURES','SECTION B: MANAGEMENT AND PROCESS DISCLOSURES','SECTION C: PRINCIPLE-WISE PERFORMANCE DISCLOSURE']
#abb sections
sections = ['SECTION A: GENERAL DISCLOSURES','SECTION B: MANAGEMENT AND PROCESS DISCLOSURES','SECTION C: PRINCIPLE WISE PERFORMANCE DISCLOSURE']

In [23]:
##Find page number in pdf using search pattern


def fnPDF_FindText(xString, startPage):
    # xfile : the PDF file in which to look
    # xString : the string to look for
    import re
    PageFound = -1
    #pdfDoc = PdfFileReader(file(xFile, "rb"))
    for i in range(startPage, num_pages):
        PgOb = reader.getPage(i)
        Text = PgOb.extractText()
        if re.search(xString,Text):
             print("String - "+xString+", found on Page: "+str(i))
             PageFound = i   
             return PageFound

In [24]:
## Get All Pages for each Section
section_pages = []
section_pages.append(3)
for i in range(0,len(sections)):
    
    start_search_string = sections[i]
    print('Start Search for section '+start_search_string)
    start_page = fnPDF_FindText(start_search_string,section_pages[i])
    section_pages.append(start_page)

    
section_pages.append(num_pages)
#remove duplicates
section_pages = [*set(section_pages)]
#sort in ascending order
section_pages.sort()
print(section_pages)

Start Search for section SECTION A: GENERAL DISCLOSURES
String - SECTION A: GENERAL DISCLOSURES, found on Page: 3
Start Search for section SECTION B: MANAGEMENT AND PROCESS DISCLOSURES
String - SECTION B: MANAGEMENT AND PROCESS DISCLOSURES, found on Page: 9
Start Search for section SECTION C: PRINCIPLE WISE PERFORMANCE DISCLOSURE
String - SECTION C: PRINCIPLE WISE PERFORMANCE DISCLOSURE, found on Page: 10
[3, 9, 10, 42]


In [25]:
## test report
test_res = camelot.read_pdf(file_path,pages='3-9')
print(len(test_res))

test_tab_res = tabula.read_pdf(file_path,pages='3-9')
print(len(test_tab_res))



2
6


In [26]:
#companyData = reader.getPage(3).extractText()
#print(companyData)

### Get data usig tables
for i in range(len(section_pages)-1):
    print('Extract for '+ sections[i])
    section_tables = camelot.read_pdf(file_path,pages=str(section_pages[i])+'-'+str(section_pages[i+1]))
    print(len(section_tables))
    output_file =  file_path[:file_path.find('.')] + '_'+ sections[i][: sections[i].find(':')]
    section_tables.export(output_file+'_camelot',f='csv',compress=True)
    tabula.convert_into(file_path, output_file+'_tabula.csv', output_format="csv", pages=list(range(section_pages[i],section_pages[i+1]+1)))

Extract for SECTION A: GENERAL DISCLOSURES
2
Extract for SECTION B: MANAGEMENT AND PROCESS DISCLOSURES
2
Extract for SECTION C: PRINCIPLE WISE PERFORMANCE DISCLOSURE
2


In [66]:
## use tabula py to extract all Section A tables in csv



## Get Employee Details - Table

In [43]:
## use camelot
#reads table from pdf file -- need to extract by providing specific areas
employeeDetails = camelot.read_pdf(file_path,pages='6')
employeeDetails[0].df


,0
0,Male\nFemale\nS. \nParticulars\nTotal (A)\nNo....
1,
2,EMPLOYEES
3,"1.\nPermanent (D)\n3,226\n2,399\n74\n 827\n26"
4,"2.\nOther than permanent (E)\n2,264\n1,120\n49..."
5,"3.\nTotal employees (D + E)\n5,490\n3,519\n64\..."


## Get CSR Details - Text

In [44]:
page_6_dt = reader.getPage(6).extractText()
csrdetails_idx=page_6_dt.find('VI. CSR Details')
compliance_idx=page_6_dt.find('VII. Transparency and Disclosures Compliance')
csrDetails = (page_6_dt[csrdetails_idx:compliance_idx])
print(csrDetails)

check_str='Whether CSR is applicable as per section 135 of Companies Act, 2013:'
isCSRApplyIndex = csrDetails.find(check_str)
csrFlag = csrDetails[isCSRApplyIndex+len(check_str)+2:isCSRApplyIndex+len(check_str)+5]
turnover_idx = csrDetails.find('(ii) Turnover: ` ')
turnover_amt_idx = csrDetails.find('crore',turnover_idx)
turnover_amt = csrDetails[turnover_idx+len('(ii) Turnover: ` '):turnover_amt_idx]
turnover_amt = int(turnover_amt.replace(',',''))
print(turnover_amt)
    
if(csrFlag.strip()=='Yes'):
    ## calculate csr amount
    print('Entering csr calculation')
    csr_text= '(iv) Total amount spent on CSR for FY21:  ` '
    csr_idx = csrDetails.find(csr_text)
    csr_amt_idx = csrDetails.find('crore',csr_idx)
    csr_amt = csrDetails[csr_idx+len(csr_text):csr_amt_idx]
    csr_amt = int(csr_amt.replace(',',''))
    print(csr_amt)
    print('Total CSR-Turnover Ratio')
    print(csr_amt/turnover_amt)
else:
    print('no csr')

VI. CSR Details
22. (i) Whether CSR is applicable as per section 135 of Companies Act, 2013:  Yes  (ii) Turnover: ` 48,176 crore (iii) Net worth:  ` 1,08,783 crore (iv) Total amount spent on CSR for FY21:  ` 190 crore

48176
Entering csr calculation
190
Total CSR-Turnover Ratio
0.003943872467618731


### Calulcate Green house gas emissions

In [45]:
envDetails = camelot.read_pdf(file_path,pages='29')
electrictyConsumption = envDetails[0].df
greenhousegases = envDetails[1].df
print(greenhousegases.head())

                                                   0  \
0                                         Parameter*   
1  Total Scope 1 emissions (Break-up of the GHG i...   
2  Total Scope 2 emissions (Break-up of the GHG i...   

                                    1        2        3  
0                                Unit     FY21     FY20  
1  Metric tonnes  of \nCO2 equivalent    966.1  1,340.1  
2  Metric tonnes  of \nCO2 equivalent  2,684.1  2,864.6  


In [46]:
pdf_file.close()